In [2]:
import os
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd

In [6]:
sf_account = os.environ['SF_ACCOUNT']
sf_password   = os.environ['SF_PASSWORD']
sf_user = os.environ['SF_USER']
database = "DEV",
schema = "CDM",
table_name = "customers"

In [26]:
df = pd.read_csv('./data/customers_.csv')
df.head()

,INDEX,CUSTOMER_ID,FIRST_NAME,LAST_NAME,COMPANY,CITY,COUNTRY,PHONE_1,PHONE_2,EMAIL,SUBSCRIPTION_DATE,WEBSITE
0,1,4962fdbE6Bfee6D,Pam,Sparks,Patel-Deleon,Blakemouth,British Indian Ocean Territory (Chagos Archipe...,267-243-9490x035,480-078-0535x889,nicolas00@faulkner-kramer.com,2020-11-29,https://nelson.com/
1,2,9b12Ae76fdBc9bE,Gina,Rocha,"Acosta, Paul and Barber",East Lynnchester,Costa Rica,027.142.0940,+1-752-593-4777x07171,yfarley@morgan.com,2021-01-03,https://pineda-rogers.biz/
2,3,39edFd2F60C85BC,Kristie,Greer,Ochoa PLC,West Pamela,Ecuador,+1-049-168-7497x5053,+1-311-216-7855,jennyhayden@petty.org,2021-06-20,https://mckinney.com/
3,4,Fa42AE6a9aD39cE,Arthur,Fields,Moyer-Wang,East Belinda,Afghanistan,001-653-754-7486x65787,521-630-3858x953,igrimes@ruiz-todd.org,2020-02-13,https://dominguez.biz/
4,5,F5702Edae925F1D,Michelle,Blevins,Shah and Sons,West Jared,Marshall Islands,8735278329,(633)283-6034x500,diamondcarter@jordan.com,2020-10-20,http://murillo-ryan.com/


In [19]:
df = df.astype('str')
# names of dataframe columns must be identical to names of columns tables in Snowflake
df.columns = [	'INDEX', 
	'CUSTOMER_ID',
	'FIRST_NAME',
	'LAST_NAME',
	'COMPANY',
	'CITY',
	'COUNTRY',
	'PHONE_1',
	'PHONE_2',
	'EMAIL',
	'SUBSCRIPTION_DATE',
	'WEBSITE'
]
df.head()

,INDEX,CUSTOMER_ID,FIRST_NAME,LAST_NAME,COMPANY,CITY,COUNTRY,PHONE_1,PHONE_2,EMAIL,SUBSCRIPTION_DATE,WEBSITE
0,1,4962fdbE6Bfee6D,Pam,Sparks,Patel-Deleon,Blakemouth,British Indian Ocean Territory (Chagos Archipe...,267-243-9490x035,480-078-0535x889,nicolas00@faulkner-kramer.com,2020-11-29,https://nelson.com/
1,2,9b12Ae76fdBc9bE,Gina,Rocha,"Acosta, Paul and Barber",East Lynnchester,Costa Rica,027.142.0940,+1-752-593-4777x07171,yfarley@morgan.com,2021-01-03,https://pineda-rogers.biz/
2,3,39edFd2F60C85BC,Kristie,Greer,Ochoa PLC,West Pamela,Ecuador,+1-049-168-7497x5053,+1-311-216-7855,jennyhayden@petty.org,2021-06-20,https://mckinney.com/
3,4,Fa42AE6a9aD39cE,Arthur,Fields,Moyer-Wang,East Belinda,Afghanistan,001-653-754-7486x65787,521-630-3858x953,igrimes@ruiz-todd.org,2020-02-13,https://dominguez.biz/
4,5,F5702Edae925F1D,Michelle,Blevins,Shah and Sons,West Jared,Marshall Islands,8735278329,(633)283-6034x500,diamondcarter@jordan.com,2020-10-20,http://murillo-ryan.com/


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 12 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   INDEX              object
 1   CUSTOMER_ID        object
 2   FIRST_NAME         object
 3   LAST_NAME          object
 4   COMPANY            object
 5   CITY               object
 6   COUNTRY            object
 7   PHONE_1            object
 8   PHONE_2            object
 9   EMAIL              object
 10  SUBSCRIPTION_DATE  object
 11  WEBSITE            object
dtypes: object(12)
memory usage: 183.1+ MB


In [7]:
cnx = snowflake.connector.connect(
    user = sf_user,
    password = sf_password,
    account = sf_account,
    database = "DEV",
    schema = "CDM",
    warehouse = "COMPUTE_WH",
    role = "ACCOUNTADMIN"
)
curs = cnx.cursor()

### Create a table 

In [15]:
curs.execute("DROP TABLE CUSTOMERS;")

create_table_query = """
CREATE TABLE IF NOT EXISTS customers (
    Index STRING,
    Customer_Id STRING,
    First_Name STRING,
    Last_Name STRING,
    Company STRING,
    City STRING,
    Country STRING,
    Phone_1 STRING,
    Phone_2 STRING,
    Email STRING,
    Subscription_Date STRING,
    Website STRING
);
"""
curs.execute(create_table_query)

### Adding data to a table

In [21]:
write_pandas(cnx,
             df,
             table_name='CUSTOMERS') # snowflake is format sensitive, table name must me in capital letters

(True,
 1,
 2000000,
 [('tbznciuosn/file0.txt',
   'LOADED',
   2000000,
   2000000,
   1,
   0,
   None,
   None,
   None,
   None)])

In [22]:
# Select data 
curs.execute("SELECT * FROM CUSTOMERS LIMIT 10;")
rows = curs.fetchall()
for row in rows:
    print(row)

('1', '4962fdbE6Bfee6D', 'Pam', 'Sparks', 'Patel-Deleon', 'Blakemouth', 'British Indian Ocean Territory (Chagos Archipelago)', '267-243-9490x035', '480-078-0535x889', 'nicolas00@faulkner-kramer.com', '2020-11-29', 'https://nelson.com/')
('2', '9b12Ae76fdBc9bE', 'Gina', 'Rocha', 'Acosta, Paul and Barber', 'East Lynnchester', 'Costa Rica', '027.142.0940', '+1-752-593-4777x07171', 'yfarley@morgan.com', '2021-01-03', 'https://pineda-rogers.biz/')
('3', '39edFd2F60C85BC', 'Kristie', 'Greer', 'Ochoa PLC', 'West Pamela', 'Ecuador', '+1-049-168-7497x5053', '+1-311-216-7855', 'jennyhayden@petty.org', '2021-06-20', 'https://mckinney.com/')
('4', 'Fa42AE6a9aD39cE', 'Arthur', 'Fields', 'Moyer-Wang', 'East Belinda', 'Afghanistan', '001-653-754-7486x65787', '521-630-3858x953', 'igrimes@ruiz-todd.org', '2020-02-13', 'https://dominguez.biz/')
('5', 'F5702Edae925F1D', 'Michelle', 'Blevins', 'Shah and Sons', 'West Jared', 'Marshall Islands', '8735278329', '(633)283-6034x500', 'diamondcarter@jordan.com',

### Dataframe from Snowflake table

In [23]:
sql = "select * from customers limit 1000;"
curs.execute(sql)
df_from_df = curs.fetch_pandas_all()
df_from_df.head()

,INDEX,CUSTOMER_ID,FIRST_NAME,LAST_NAME,COMPANY,CITY,COUNTRY,PHONE_1,PHONE_2,EMAIL,SUBSCRIPTION_DATE,WEBSITE
0,1,4962fdbE6Bfee6D,Pam,Sparks,Patel-Deleon,Blakemouth,British Indian Ocean Territory (Chagos Archipe...,267-243-9490x035,480-078-0535x889,nicolas00@faulkner-kramer.com,2020-11-29,https://nelson.com/
1,2,9b12Ae76fdBc9bE,Gina,Rocha,"Acosta, Paul and Barber",East Lynnchester,Costa Rica,027.142.0940,+1-752-593-4777x07171,yfarley@morgan.com,2021-01-03,https://pineda-rogers.biz/
2,3,39edFd2F60C85BC,Kristie,Greer,Ochoa PLC,West Pamela,Ecuador,+1-049-168-7497x5053,+1-311-216-7855,jennyhayden@petty.org,2021-06-20,https://mckinney.com/
3,4,Fa42AE6a9aD39cE,Arthur,Fields,Moyer-Wang,East Belinda,Afghanistan,001-653-754-7486x65787,521-630-3858x953,igrimes@ruiz-todd.org,2020-02-13,https://dominguez.biz/
4,5,F5702Edae925F1D,Michelle,Blevins,Shah and Sons,West Jared,Marshall Islands,8735278329,(633)283-6034x500,diamondcarter@jordan.com,2020-10-20,http://murillo-ryan.com/
